In [294]:
from numba import njit,jit,vectorize,prange
from numba import cuda
import numpy as np
#import struct
import random
import time
import tensorflow as tf

In [295]:
def preprocessing(tx_train, ty_train, tx_test, ty_test):
        
    py_train = ty_train.flatten()
    py_test = ty_test.flatten()
    
    px_test = tx_test.reshape(10000,input_size)
    px_train = tx_train.reshape(60000,input_size)
    
    px_train = np.asarray(px_train).astype(np.int32)
    py_train = np.asarray(py_train).astype(np.int32)
    px_test = np.asarray(px_test).astype(np.int32)
    py_test = np.asarray(py_test).astype(np.int32)

    return px_train, py_train, px_test, py_test 

In [283]:
class WiSARD:                                                     
    
    def __init__(self,input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number):
        self.input_size = input_size
        self.no_of_rand_pix_selec = no_of_rand_pix_selec
        self.nodes = nodes
        self.ram_address_count = ram_address_count
        self.dis_number = dis_number 


    def discriminator(self):
        discriminator = []
        accumulated_pos = []
        my_list = list(range(0,self.input_size))
        for i in range(self.dis_number):  #10
            ram = []
            random.shuffle(my_list)
            for j in range((int)((self.nodes))): #98    
                total_pos = []            
                positions = []
                positions = my_list[j*no_of_rand_pix_selec:j*no_of_rand_pix_selec+no_of_rand_pix_selec]
                accumulated_pos.append(positions)
                total_pos = np.vstack(positions)
                #print(total_pos)
                table = []
                dictionary = {}
                
                max = len("{0:b}".format(2**len(total_pos))) - 1
                for i in range(2**len(total_pos)):
                    x = (('0' * max) + "{0:b}".format(i))
                    x = x[len(x)-max:]
                    dictionary[x] = 0
                table.append(dictionary)
                #print(table)
                ram.append(table)
        
            di = []
            for j in range(len(ram)):
                for i in range(len(ram[j])):
                    for key, value in ram[j][i].items():
                        temp = [key,value]
                        di.append(temp)
                        
            discriminator.append(di)
    
        discriminator = np.asarray(discriminator).astype(np.int32)
        accumulated_pos = np.asarray(accumulated_pos).astype(np.int32)
        return discriminator, accumulated_pos
        
    
    @staticmethod
    @njit(parallel = True)
    def train(d,pos,x_train, y_train):
        
        images = x_train
        lable = y_train    
        
        for i in range(len(images)):
            image = images[i]
            num = lable[i]
            all_ram_of_selected_discriminator = d[num]
            t_ratina = pos[(int)(nodes*num):(int)(nodes*num+nodes)]
            
            for i in range((int)(nodes)):
                part = all_ram_of_selected_discriminator[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
                ratina_for_one_ram = t_ratina[i]               
                n = []                                                                
                for ix in prange(len(ratina_for_one_ram)):
                    pix = ratina_for_one_ram[ix]
                    if image[(pix-1)]>=1:
                        n.append(1)     
                    else:
                        n.append(0)
                
                num = 0
                for i in range(no_of_rand_pix_selec):
                    num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
            
                address_of_that_ram = (int)(num)
                for key in prange(ram_address_count):
                    index = part[key]          
                    if index[0] == address_of_that_ram:
                        index[1] = 1                        
                        
    
    @staticmethod
    @njit(parallel = True)
    def train_with_bleeching(d,pos,x_train, y_train):
        
        images = x_train
        lable = y_train    
        
        for i in range(len(images)):
            image = images[i]
            num = lable[i]
            all_ram_of_selected_discriminator = d[num]
            t_ratina = pos[(int)(nodes*num):(int)(nodes*num+nodes)]
            
            for i in range((int)(nodes)):
                part = all_ram_of_selected_discriminator[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
                ratina_for_one_ram = t_ratina[i]               
                n = []                                                                
                for ix in range(len(ratina_for_one_ram)):
                    pix = ratina_for_one_ram[ix]
                    if image[(pix-1)]>=1:
                        n.append(1)    
                    else:
                        n.append(0)
                
                num = 0
                for i in range(no_of_rand_pix_selec):
                    num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
            
                address_of_that_ram = (int)(num)
                for key in range(ram_address_count):
                    index = part[key]          
                    if index[0] == address_of_that_ram:
                        index[1] += 1
            


    @staticmethod
    @njit(parallel = True)
    def test(d,pos,x_test,y_test):
        right = 0
        wrong = 0
        images = x_test
        lable = y_test
        
        for i in range(len(images)):
            image = images[i]
            actual_lable = lable[i]
            
            total_sum=[]
            
            for ix in prange(dis_number):
                
                t_ratina = pos[int((nodes*ix)):(int)((nodes*ix+nodes))]
                
                sum_of_ram_output = 0
                dis = d[ix]
                
                for i in range(int(nodes)):
                    part = dis[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
                    ratina_for_one_ram = t_ratina[i]
                    
                    n = []                                                                
                    for pix in ratina_for_one_ram:
                        if image[(pix-1)]>=1:
                            n.append(1)
                        else:
                            n.append(0)
                    
                    num = 0
                    for i in prange(no_of_rand_pix_selec):
                        num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
                    
                    address_of_that_ram = (int)(num)
                
                    for key in prange(len(part)):
                        prt = part[key]
                        if prt[0] == address_of_that_ram and prt[1]>=1:
                            sum_of_ram_output += 1
                
                total_sum.append(sum_of_ram_output)        
        
            max_sum = 0
            idx = 0
            for i in prange(len(total_sum)):
                if max_sum < total_sum[i]:
                    max_sum = total_sum[i]
                    idx = i
            index_of_dis = idx
            if index_of_dis == actual_lable:
                right += 1
            else:
                wrong += 1
                
        return right,wrong




    #@vectorize(['int32(int32,int32,int32,int32)'], target = 'cuda')
    @staticmethod
    @njit(parallel = True)
    #@cuda.jit
    def test_with_bleaching(d,pos,x_test,y_test):
        right = 0
        wrong = 0
        images = x_test
        lable = y_test
        ct = 0.015
        b=1
        
        for i in range(len(images)):
            image = images[i]
            actual_lable = lable[i]
            
            total_sum=[]
            
            for ix in range(dis_number):
                
                t_ratina = pos[int((nodes*ix)):(int)((nodes*ix+nodes))]
                
                sum_of_ram_output = 0
                dis = d[ix]
                
                for i in range(int(nodes)):
                    part = dis[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
                    ratina_for_one_ram = t_ratina[i]
                    
                    n = []                                                                
                    for pix in ratina_for_one_ram:
                        if image[(pix-1)]>=1:
                            n.append(1)
                        else:
                            n.append(0)
                    
                    num = 0
                    for i in range(no_of_rand_pix_selec):
                        num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
                    
                    address_of_that_ram = (int)(num)
                
                    for key in range(len(part)):
                        prt = part[key]
                        if prt[0] == address_of_that_ram and prt[1]>=b:           
                            sum_of_ram_output += 1
                
                total_sum.append(sum_of_ram_output)        
        
            max_sum = 0
            sec_max = 0
            idx = 0
            
            for i in range(len(total_sum)):
                if max_sum < total_sum[i]:
                    max_sum = total_sum[i]
                    idx = i
                    
            for j in range(len(total_sum)):
                if sec_max < total_sum[j] and j!=idx:
                    sec_max = total_sum[j]
                    
            index_of_dis = idx
            if index_of_dis == actual_lable:
                right += 1
            else:
                wrong += 1
            
            if max_sum == sec_max or max_sum == 0:
                confidence = 0
            else:
                confidence = 1 - float(sec_max)/float(max_sum)
            if confidence < ct:
                b += 1
        
        return right,wrong

In [284]:
input_size = 28*28
no_of_rand_pix_selec = 2**(3)     ## ** (must) no_of_rand_pix_selec = 2^(n) where n is 0,1,2...
nodes = int(input_size/no_of_rand_pix_selec)    #98
ram_address_count = 2**(no_of_rand_pix_selec)   #256
dis_number = 10 

In [285]:
(tx_train, ty_train), (tx_test, ty_test) = tf.keras.datasets.mnist.load_data()
px_train, py_train, px_test, py_test = preprocessing(tx_train, ty_train, tx_test, ty_test)

In [286]:
w = WiSARD(input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number)

In [287]:
d, acc_pos = w.discriminator()

In [288]:
starttrain = time.time()
w.train_with_bleeching(d,acc_pos,px_train[0:60000],py_train[0:60000])
endtrain = time.time()
print("time train = ",endtrain - starttrain)

time train =  18.838600635528564


In [289]:
starttest = time.time()
right,wrong = w.test_with_bleaching(d,acc_pos,px_test[0:10000],py_test[0:10000])
endtest = time.time()
print("time test = ",endtest - starttest)

time test =  31.629737377166748


In [290]:
print("number of right result = ",right)
print("number of wrong results = ",wrong)
accuracy = ((right)/(right+wrong))*100
print("accuracy by testing the model =",accuracy)

number of right result =  8578
number of wrong results =  1422
accuracy by testing the model = 85.78


In [271]:
nsamples = 10000000

In [281]:
import random

def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

@jit
def monte_carlo_pi_jit(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

@cuda.jit
def monte_carlo_pi_njit(nsamples):
    acc = 0
    for i in range(nsamples):
        x = 2
        y = 3
        if (x**2 + y**2) < 1.0:
            acc += 1
    return (acc / nsamples)

In [282]:
start = time.time()
monte_carlo_pi_njit(nsamples)
end = time.time()
print("time = ",end - start)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No conversion from float64 to none for '$58.4', defined at None

File "<ipython-input-281-c029cdb6fa35>", line 30:
def monte_carlo_pi_njit(nsamples):
    <source elided>
            acc += 1
    return (acc / nsamples)
    ^

[1] During: typing of assignment at <ipython-input-281-c029cdb6fa35> (30)

File "<ipython-input-281-c029cdb6fa35>", line 30:
def monte_carlo_pi_njit(nsamples):
    <source elided>
            acc += 1
    return (acc / nsamples)
    ^
